In [15]:
from collections import OrderedDict
import sympy 
from sympy.parsing.sympy_parser import parse_expr


## PART-1

In [16]:
def bisection_method(func,interval,iterations=5):
    a,b=interval
    if(func.evaluate(a)*func.evaluate(b)<0):
        if(func.evaluate(a)>0):
            pl=a
            nl=a
        else:
            pl=b
            nl=a
        
        for ite in range(iterations):
            print(f'iteration %d: \nvalue of pl=%f \nvalue of nl=%f'%(ite+1,pl,nl))
            x=float(pl+nl)/2.0
            value=func.evaluate(x)
            if(value==0):
                root=0
                break
            else:
                if(value<0):
                    nl=x
                else:
                    pl=x
            print('x = %f'%x)
            print('\n')
        root=x
        print('root is %f'%root)
        print('\n')
        return root
        
    else:
        raise Exception('working condition of bisection method not satified.')
        
def regula_falsi_method(func,interval,iterations=5):
    a,b=interval
    if(func.evaluate(a)*func.evaluate(b)<0):
        if(func.evaluate(a)>0):
            pl=a
            nl=a
        else:
            pl=b
            nl=a
            
        for ite in range(iterations):
            print(f'iteration %d: \nvalue of pl=%f \nvalue of nl=%f'%(ite+1,pl,nl))
            x=nl-(func.evaluate(nl)*(pl-nl))/(func.evaluate(pl)-func.evaluate(nl))
            if func.evaluate(x)<0:
                nl=x
            else:
                pl=x
            print('x = %f'%x)
            print('\n')
            
        root=x
        print('root is %f'%root)
        print('\n')
        return root
    else:
        raise Exception('working condition of bisection method not satified.')
        
def secant_method(func,initials,iterations=5):
    x0,x1=initials
    print("Second Method")
    for ite in range(iterations):
        print(f'iteration %d: \nvalue of x0=%f \nvalue of x1=%f'%(ite+1,x0,x1))
        x=x1-(func.evaluate(x1)*(x1-x0))/(func.evaluate(x1)-func.evaluate(x0))
        x0=x1
        x1=x
        print('x = %f'%x)
        print('\n')
    
    root=x
    print('root is %f'%root)
    print('\n')
    return root
        
def newton_raphson_method_v1(func,initial_approx,iterations=5):
    x0=initial_approx
    x=None
    for ite in range(iterations):
        print('iteration %d: \nvalue of x%d=%f'%(ite+1,ite,x0) )
        x=x0-func.evaluate(x0)/(func.diff(times=1,subs=x0))
        print('value of x%d=%f'%(ite+1,x))
        x0=x
        print('\n')
        
    root=x
    print('root is %f'%root)
    print('\n')
    return root

class OneUnknownFunc:
    
    def __init__(self,definition):
        if isinstance(definition,str):
            self.definition=parse_expr(definition)
        else:   
            self.definition=definition
        self.var=list(self.definition.free_symbols)[0]

    def evaluate(self,subs):
        result=self.definition.subs(self.var,subs)
        return result
    
    def diff(self,times=1,subs=None):
        deriv=OneUnknownFunc(sympy.diff(self.definition,self.var,times))
        if subs==None:
            return deriv
        else:
            return deriv.evaluate(subs)
    
    def bisection_meth(self,interval,iterations=5):
        root=bisection_method(self,interval,iterations)
        return root
    
    def regula_falsi_meth(self,interval,iterations=5):
        root=regula_falsi_method(self,interval,iterations)
        return root
    
    def secant_meth(self,initials,iterations=5):
        root=secant_method(self,initials,iterations)
        return root
    
    def newton_raphson_meth_v1(self,initial_approx,iterations=5):
        root=newton_raphson_method_v1(self,initial_approx,iterations)
        return root
    

## PART-1: How to use the methods

In [17]:
#validation: OneUnknownFunc
#func1=OneUnknownFunc('x**3-4*x-8')
#func1.bisection_meth((2,3),4)

#func2=OneUnknownFunc('x**3-9*x+2')
#func2.regula_falsi_meth((2,3),5)

#func3=OneUnknownFunc('x**3+x**2+x+6')
#func3.secant_meth((-3,-1))

func4=OneUnknownFunc('3*x-cos(x)-2')
func4.newton_raphson_meth_v1(0.6)

iteration 1: 
value of x0=0.600000
value of x1=0.887641


iteration 2: 
value of x1=0.887641
value of x2=0.879250


iteration 3: 
value of x2=0.879250
value of x3=0.879244


iteration 4: 
value of x3=0.879244
value of x4=0.879244


iteration 5: 
value of x4=0.879244
value of x5=0.879244


root is 0.879244




0.879244435158154

## PART-2

In [18]:
class MultipleUnknownFunc:
    def __init__(self, definition):
        if isinstance(definition,str):
            self.definition=parse_expr(definition)
        else:
            self.definition=definition
        self.variables=list(self.definition.free_symbols)
        
    def get_definition(self):
        return self.definition
    
    def evaluate(self,values):
        #values is a dictionary with variables (either valid symbols or string) as keys and values as values
        return self.definition.subs(values)
    
    def diff(self,*args,subs=None):
        deriv=MultipleUnknownFunc(sympy.diff(self.definition,*args))
        if(subs==None):
            return deriv
        else:
            return deriv.evaluate(subs)
        
    

In [19]:
class SystemOfEquations:
    def __init__(self,dim,var_list,definitions):
        self.dim=dim
        self.var_list=var_list
        self.functions=[]
        self.func_definitions_sympy=[]
        for f in definitions:
            if isinstance(f,str):
                sympy_ex=parse_expr(f)
            else:
                sympy_ex=f
            self.func_definitions_sympy.append(sympy_ex)
            self.functions.append(MultipleUnknownFunc(sympy_ex))
        
    def evaluate_the_functions(self,values):
        result=[]
        for f in self.functions:
            #values is a dictionary with variables (either valid symbols or string) as keys and values as values
            result.append(f.evaluate(values))
        return result

    def get_J(self,subs=None):
        list2=[]
        for f in self.functions:
            list1=[]
            for var in self.var_list:
                result=f.diff(var)
                list1.append(result.get_definition())
            list2.append(list1)
        if subs==None:
            return sympy.Matrix(list2)
        else:
            return sympy.Matrix(list2).subs(subs)
    
    def get_J_inv(self,subs=None):
        if subs==None:
            return self.get_J().inv()
        else:
            return self.get_J().inv().subs(subs)
        
    def get_F(self,subs=None):
        if subs==None:
            return sympy.Matrix(self.func_definitions_sympy)
        else:
            return sympy.Matrix(self.func_definitions_sympy).subs(subs)
        
    def newton_raphson_meth(self,initial_approx,iterations=5):
        #initial_approx is a list with variable-value pair tuples. It is use to create an OrderedDict,which has the same name.
        initial_approx=OrderedDict(initial_approx)
        initial_approx_mat=sympy.Matrix(list(initial_approx.values()))
        for ite in range(iterations):
            print("iteration %d:"%(ite+1))
            print("X%d : "%(ite),initial_approx_mat)
            solution = initial_approx_mat - self.get_J_inv(subs=initial_approx)*self.get_F(subs=initial_approx)
            initial_approx_mat=solution
            # update the initial_approx dictionary
            for k,nval in zip(initial_approx,list(solution)):
                initial_approx[k]=nval
                
            
        return solution
            
        
    
    

## PART-2: How to use the methods

In [20]:
functions=['x**2+x*y+y**2-19','x**3+y**3-35']
sys1=SystemOfEquations(2,['x','y'],functions)
initial_approx=[('x',4.91724),('y',-0.38578)]
sys1.newton_raphson_meth(initial_approx)

iteration 1:
X0 :  Matrix([[4.91724000000000], [-0.385780000000000]])
iteration 2:
X1 :  Matrix([[3.75018093977353], [1.44651008595981]])
iteration 3:
X2 :  Matrix([[3.20687242215275], [1.78971641724617]])
iteration 4:
X3 :  Matrix([[3.03056617394911], [1.96946277746900]])
iteration 5:
X4 :  Matrix([[3.00087984975417], [1.99912012087948]])


Matrix([
[3.00000077279568],
[1.99999922720516]])

## PART-3

In [21]:
import numpy as np
class SystemOfLinearEq:
    def __init__(self,A,B):
        self.A=A
        self.B=B
        self.size=len(self. B)
        
    def summ(self,i,initial_vals):
        summ=0
        for x in range(1,self.size):
            j=(i+x)%self.size
            summ+=self.A[i][j]*initial_vals[j]
            #print("a[i][j]=",self.A[i][j],"init[j]=",initial_vals[j],"i=",i,"j=",j,"x=",x)
        return summ    
            
        
    def gauss_jacobi_elementwise_meth(self,initial_vals,iterations=5):
        initial_vals_list=initial_vals.copy()
        for ite in range(iterations):
            approx=[]
            for i in range(self.size):
                x=(self.B[i]-(self.summ(i,initial_vals_list)))/self.A[i][i]
                approx.append(x)
            initial_vals_list=approx.copy()
            print("Iteration {0} : Xapprox={1}".format(ite+1,initial_vals_list))
            print("\n")
                
    def gauss_siedel_matrix_meth(self,initial_vals,iterations=5):
        X=np.array(initial_vals)
        A=np.array(self.A)
        Lx=np.tril(A,0)
        B=np.array(self.B)
        U=np.triu(A,1)
        
        for ite in range(iterations):
            #print("X",X,"\nU",U)
            ux=np.matmul(U,X)
            #print("ux",ux)
            Xapp=np.matmul(np.linalg.inv(Lx),B-ux)
            #print("xapp",Xapp)
            X=Xapp.copy()
            print("Iteration {0} : Xapprox={1}".format(ite+1,list(X)))
            print("\n")
            

## PART-3: How to use the methods

In [22]:
s1=SystemOfLinearEq([[4,1,0],[1,3,-1],[0,-1,4]],[3,4,5])
s1.gauss_siedel_matrix_meth([0,0,0],4)

Iteration 1 : Xapprox=[0.75, 1.0833333333333333, 1.5208333333333333]


Iteration 2 : Xapprox=[0.4791666666666667, 1.6805555555555554, 1.6701388888888888]


Iteration 3 : Xapprox=[0.32986111111111116, 1.7800925925925926, 1.6950231481481481]


Iteration 4 : Xapprox=[0.30497685185185186, 1.796682098765432, 1.699170524691358]




In [ ]:
s1=SystemOfLinearEq([[7,2,1],[-3,8,3],[1,1,-6]],[10,-14,-33])
s1.gauss_siedel_matrix_meth([1,1,1],4)

s1=SystemOfLinearEq([[4,1,2],[3,4,1],[1,2,3]],[4,7,3])
s1.gauss_siedel_matrix_meth([1,1,1],4)

s1=SystemOfLinearEq([[10,-2,1],[-2,10,-2],[-2,-5,10]],[9,12,18])
s1.gauss_siedel_matrix_meth([0.5,0.5,0.5],4)

s1=SystemOfLinearEq([[8,1,-1],[-1,7,-2],[2,1,9]],[8,4,12])
s1.gauss_siedel_matrix_meth([0,0,0],4)

s1=SystemOfLinearEq([[5,-1,0],[-1,5,-1],[0,-1,5]],[9,4,-6])
s1.gauss_siedel_matrix_meth([0,0,0],4)

s1=SystemOfLinearEq([[4,1,0],[1,3,-1],[1,0,2]],[3,-4,5])
s1.gauss_siedel_matrix_meth([0,0,0],4)